In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
df_land = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/Land_Cover.csv')

df_socioecon = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/usa_00008(3).csv')

df_politics = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/countypres_2000-2020.csv')

df_religion = pd.read_excel('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/2020_USRC_Summaries.xlsx', sheet_name='2020 Metro Summary')

# Walking score may need different data

# Transit quality:

folder_path = 'C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/Transit_Qual'
dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # or '.xls' if your files are in the older Excel format
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the Excel file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all the DataFrames in the list into a single DataFrame
df_transit = pd.concat(dfs, ignore_index=True)

df_marijuana = pd.read_excel('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/marijuana.xlsx')

cbsa_to_fips = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/cbsa2fipsxw.csv')

In [3]:
df_land.head(2)

,FIPS,State,County,Year,Total Accounted,Major Crops,Minor Crops,Pasture,Hay,Forest,Developed,Developed Open,Developed Low,Developed Med,Developed High
0,1001,Alabama,Autauga County,2017,379685.68,11944.59,1850.54,20072.88,33499.73,289383.06,22934.88,15903.21,4924.93,1663.29,443.45
1,1001,Alabama,Autauga County,2010,379826.90,12540.83,1492.27,53223.45,21882.95,271317.95,19369.45,13842.28,4108.52,1116.20,302.46


In [4]:
df_socioecon.head(2)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,CITYPOP,STRATA,...,SEX,AGE,RACE,RACED,RACHSING,EDUC,EDUCD,EMPSTAT,EMPSTATD,IND
0,2021,202101,1,2021010000026,13.0,2021000000011,1,0,0,80001,...,1,85,1,100,1.0,7,71,3,30,0
1,2021,202101,2,2021010000031,51.0,2021000000021,1,0,0,80001,...,2,67,2,200,2.0,6,63,3,30,0


In [5]:
df_politics.head(2)

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL


In [6]:
df_religion.head(2)

,CBSA Code,Metro Name,2020 Population,Congregations,Adherents,"Congregations per 100,000 Population",Adherents as % of Population,Population Rank,Congregations Rank,Adherents Rank,"Congregations Per 100,000 Pop. Rank",Adherents as % of Population Rank
0,10100,"Aberdeen, SD Micro Area",42287.0,101.0,31367.0,238.844089,0.741765,672.0,553.0,510.0,158.0,41.0
1,10140,"Aberdeen, WA Micro Area",75636.0,92.0,21494.0,121.635200,0.284177,463.0,597.0,648.0,639.0,879.0


In [7]:
df_transit.head(2)

,place,name,blkgrps,population,households,alltransit_performance_score
0,"""0100100""","""Abanda""",0.03,40.96,15.98,0.0
1,"""0100124""","""Abbeville""",1.34,1282.40,476.84,0.0


In [8]:
df_marijuana.head(2)

,State,Legal Status,Medicinal,Decriminalized
0,Alabama,Mixed,Yes,No
1,Alaska,Fully Legal,Yes,Yes


In [9]:
# 1. df_land should be year = 2022 only
# 2. df_socioecon should be Year = 2022 only and drop column RACHSING
# 3. df_socioecon needs FIPS created, with leading zeros restored in STATEFIP and COUNTYFIP and then combined
# 4. does population work?
# 5. df_politics needs county_fips to have decimal dropped
# 6. df_religion needs cbsa converted to fips using cbsa_to_fips
# 7. df_transit needs place renamed FIPS, with the "" removed
# 8. df_marijuana needs STATEFIPS made based on State

In [10]:
df_land_2022 = df_land[df_land['Year'] == 2022]  #1
df_land_2022 = df_land_2022.rename(columns={'Year': 'YEAR'})

In [11]:
df_socioecon_2022 = df_socioecon[df_socioecon['YEAR'] == 2022]  #2
df_socioecon_2022 = df_socioecon_2022.drop(columns={'RACHSING'})

In [12]:
# Ensure the STATEFIP and COUNTYFIP columns are strings with leading zeros as needed  #3
# We assume 2 characters for state FIPS and 3 for county FIPS as a common case
df_socioecon_2022['STATEFIP_str'] = df_socioecon_2022['STATEFIP'].apply(lambda x: str(x).zfill(2))
df_socioecon_2022['COUNTYFIP_str'] = df_socioecon_2022['COUNTYFIP'].apply(lambda x: str(x).zfill(3))

# Combine the formatted STATEFIP and COUNTYFIP columns to create the FIPS code
df_socioecon_2022['FIPS'] = df_socioecon_2022['STATEFIP_str'] + df_socioecon_2022['COUNTYFIP_str']

# Optionally, you can drop the intermediate string columns if they are no longer needed
df_socioecon_2022.drop(columns=['STATEFIP_str', 'COUNTYFIP_str'], inplace=True)

df_socioecon_2022.head(2)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,CITYPOP,STRATA,...,SEX,AGE,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,IND,FIPS
3252599,2022,202201,1,2022010000031,69.0,2022000000011,1,97,0,280301,...,2,85,8,801,7,71,3,30,0,01097
3252600,2022,202201,2,2022010000111,22.0,2022000000021,1,0,0,200001,...,1,51,1,100,6,64,3,30,5391,01000


In [13]:
df_socioecon_2022['CITYPOP'].describe()  #4

count    3.373378e+06
mean     2.638443e+03
std      1.244737e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.335900e+04
Name: CITYPOP, dtype: float64

In [14]:
# Filter out zero values in the CITYPOP column and then find the minimum
min_citypop_excluding_zeros = df_socioecon_2022[df_socioecon_2022['CITYPOP'] != 0]['CITYPOP'].min()

print(min_citypop_excluding_zeros)

# This has population of all cities but counties should be added too:
df_countypop = pd.read_csv('C:/Users/benja/OneDrive - University of Tennessee/DATA_499/data/COUNTY_POP.csv')

1008


In [15]:
# Ensure the FIPS column is a string and has leading zeros to make it 5 digits long
df_countypop['FIPS'] = df_countypop['FIPS'].apply(lambda x: str(x).zfill(5))

df_countypop['COUNTY_POP'].describe()

count    2.991000e+03
mean     1.064829e+05
std      3.385943e+05
min      5.100000e+01
25%      1.077000e+04
50%      2.594000e+04
75%      6.799350e+04
max      9.721138e+06
Name: COUNTY_POP, dtype: float64

In [16]:
# Ensure county_fips is treated as a string, and replace the decimal point  #5
df_politics['county_fips'] = df_politics['county_fips'].astype(str).str.replace('.', '')

# If the operation may result in strings longer than 5 digits, truncate to the first 5 digits
df_politics['county_fips'] = df_politics['county_fips'].str[:5]

In [17]:
# Ensure both 'CBSA Code' in df_religion and 'cbsacode' in cbsa_to_fips are strings before merging  #6
df_religion['CBSA Code'] = df_religion['CBSA Code'].astype(str)
cbsa_to_fips['cbsacode'] = cbsa_to_fips['cbsacode'].astype(str)

# Now perform the merge
df_religion = pd.merge(df_religion, cbsa_to_fips, left_on='CBSA Code', right_on='cbsacode', how='left')

# Function to safely convert codes to string with leading zeros, handling NaN values
def safe_format_code(code, desired_length):
    if pd.isna(code):
        return np.nan  # Return NaN as is
    # Convert to int to remove any decimals, then format with leading zeros
    return '{:0>{width}}'.format(int(float(code)), width=desired_length)

# Convert fipsstatecode and fipscountycode to strings without decimals and ensure leading zeros
df_religion['fipsstatecode'] = df_religion['fipsstatecode'].apply(lambda x: safe_format_code(x, 2))
df_religion['fipscountycode'] = df_religion['fipscountycode'].apply(lambda x: safe_format_code(x, 3))

# Concatenate the cleaned state and county codes to create the FIPS code
df_religion['FIPS'] = df_religion['fipsstatecode'] + df_religion['fipscountycode']

# Optionally, drop the original state and county code columns if they are no longer needed
df_religion.drop(columns=['fipsstatecode', 'fipscountycode'], inplace=True)

df_religion.head(2)

,CBSA Code,Metro Name,2020 Population,Congregations,Adherents,"Congregations per 100,000 Population",Adherents as % of Population,Population Rank,Congregations Rank,Adherents Rank,...,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,centraloutlyingcounty,FIPS
0,10100,"Aberdeen, SD Micro Area",42287.0,101.0,31367.0,238.844089,0.741765,672.0,553.0,510.0,...,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,Central,46013
1,10100,"Aberdeen, SD Micro Area",42287.0,101.0,31367.0,238.844089,0.741765,672.0,553.0,510.0,...,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,Outlying,46045


In [18]:
# Rename the 'place' column to 'FIPS'  #7
df_transit.rename(columns={'place': 'FIPS'}, inplace=True)

# Remove double quotes from the 'FIPS' column values
df_transit['FIPS'] = df_transit['FIPS'].str.replace('"', '')

# Verify the changes
df_transit.head(2)

# FIPS is suspicious

,FIPS,name,blkgrps,population,households,alltransit_performance_score
0,0100100,"""Abanda""",0.03,40.96,15.98,0.0
1,0100124,"""Abbeville""",1.34,1282.40,476.84,0.0


In [19]:
state_fips_dict = {
    'Alabama': '01', 'Alaska': '02', 'Arizona': '04', 'Arkansas': '05',
    'California': '06', 'Colorado': '08', 'Connecticut': '09', 'Delaware': '10',
    'District of Columbia': '11', 'Florida': '12', 'Georgia': '13', 'Hawaii': '15',
    'Idaho': '16', 'Illinois': '17', 'Indiana': '18', 'Iowa': '19',
    'Kansas': '20', 'Kentucky': '21', 'Louisiana': '22', 'Maine': '23',
    'Maryland': '24', 'Massachusetts': '25', 'Michigan': '26', 'Minnesota': '27',
    'Mississippi': '28', 'Missouri': '29', 'Montana': '30', 'Nebraska': '31',
    'Nevada': '32', 'New Hampshire': '33', 'New Jersey': '34', 'New Mexico': '35',
    'New York': '36', 'North Carolina': '37', 'North Dakota': '38', 'Ohio': '39',
    'Oklahoma': '40', 'Oregon': '41', 'Pennsylvania': '42', 'Rhode Island': '44',
    'South Carolina': '45', 'South Dakota': '46', 'Tennessee': '47', 'Texas': '48',
    'Utah': '49', 'Vermont': '50', 'Virginia': '51', 'Washington': '53',
    'West Virginia': '54', 'Wisconsin': '55', 'Wyoming': '56'
}  #8

# Assuming df_marijuana is your DataFrame and it has a column named 'State'
# Map the state names to their FIPS codes
df_marijuana['STATEFIPS'] = df_marijuana['State'].map(state_fips_dict)

# Check the first few rows to verify the changes
df_marijuana.head(2)

,State,Legal Status,Medicinal,Decriminalized,STATEFIPS
0,Alabama,Mixed,Yes,No,01
1,Alaska,Fully Legal,Yes,Yes,02


In [21]:
# combine!!

# df_koob =